In [90]:
#Import Lib
import pandas as pd
from bs4 import BeautifulSoup
import nltk
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import json
import re
from nltk.sentiment import SentimentIntensityAnalyzer
import spacy
from collections import defaultdict, Counter
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
import swifter
from sklearn.model_selection import train_test_split
from scipy.sparse import hstack
import warnings
warnings.filterwarnings("ignore")
from wordcloud import WordCloud,STOPWORDS
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, fpmax, fpgrowth


color_pal = sns.color_palette()
nlp = spacy.load('en_core_web_sm')
sia = SentimentIntensityAnalyzer()

In [4]:
## We will load this when we have json structure. For now to start coding we will use from csv
# df = pd.read_json("nlp_questions.json") 

df = pd.read_csv("QueryResults.csv")


In [5]:
df

,QuestionId,Title,Body,CreationDate,Score,ViewCount,AnswerCount,AllTags,AcceptedAnswerId,AcceptedAnswerBody
0,79501178,Store images instead of showing in a server,<p>I am running the code found on this [site][...,2025-03-11 14:50:31,0,23,1,<python><nlp><large-language-model>,79501337.0,<p>I can't test it but ...</p>\n<p>I checked <...
1,79498915,Comparing the similarity of spoken and written...,<p>I'm converting spoken form text to its writ...,2025-03-10 18:55:59,0,20,1,<nlp><large-language-model>,NaN,NaN
2,79488426,Upserting in Pinecone takes too long,<p>I'm trying to upsert reviews that i've scra...,2025-03-06 06:22:35,1,37,1,<python><nlp><rag><pinecone>,NaN,NaN
3,79484448,How does ELMo generate words for training ? Is...,<p>I'm confused about using Bidirectional LM f...,2025-03-04 17:32:14,0,28,1,<nlp><language-model><autoregressive-models><e...,NaN,NaN
4,79482290,How to handle German language specific charact...,"<p>I am working with German Texts, where I nee...",2025-03-03 22:32:36,1,59,1,<python><nlp><tokenize><large-language-model><...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
16674,62328,Is there an algorithm that tells the semantic ...,"<p>input: phrase 1, phrase 2</p>\n\n<p>output:...",2008-09-15 12:26:42,65,49872,11,<algorithm><nlp><semantics>,63076.0,<hr>\n\n<p>You might want to check out this pa...
16675,41424,"How do you implement a ""Did you mean""?",<blockquote>\n <p><strong>Possible Duplicate:...,2008-09-03 10:36:13,118,33149,17,<nlp>,41448.0,<p>Actually what Google does is very much non-...
16676,36533,Vista speech recognition in multiple languages,"<p>my primary language is spanish, but I use a...",2008-08-31 01:08:48,3,5658,6,<windows-vista><nlp><speech-recognition><multi...,36684.0,"<p>Citation from Vista <a href=""http://blogs.m..."
16677,25332,What's a good natural language library to use ...,<p>I'm looking for an existing library to summ...,2008-08-24 20:57:33,14,6483,4,<language-agnostic><nlp>,NaN,NaN


In [6]:

filtered_df = df.loc[df['AcceptedAnswerBody'].str.contains('<img', na=False), 'AcceptedAnswerBody']
print(filtered_df)

79       <p><a href="https://huggingface.co/docs/transf...
195      <h3>Contextual embedding models are computatio...
209      <p>So found this in StanfordNLP Javadoc of <em...
405      <p>Latest release of deepspeech was 3 years ba...
423      <p>The first issue with your code is that you ...
                               ...                        
15613    <p>If all you want is adjective frequencies, t...
15804    <p>The type of plot you referred to in the OP ...
15816    <p>This problem is not that stupid. Norvig wro...
15858    <p>For example - you might use some classes fr...
16478    <p>The basic idea of doing something like this...
Name: AcceptedAnswerBody, Length: 238, dtype: object


### Generate graphs using popular python libraries to visualise the data.

## Preporcess Data

In [7]:
intensifiers = {
    "very", "really", "extremely", "absolutely", "totally", "highly", "deeply", 
    "strongly", "incredibly", "exceptionally", "remarkably", "unbelievably", 
    "insanely", "awfully", "horribly", "hugely", "immensely", "overly", 
    "particularly", "significantly", "seriously", "tremendously", "wildly",
    "super", "ultra", "crazy", "majorly"
}
def clean_text(text,remove_code=True):

    # Check if text is None, empty, or NaN
    if text is None or text == "" or (isinstance(text, float) and np.isnan(text)):
        return ""
    # Convert to string if it's not already (handles numbers, etc.)
    if not isinstance(text, str):
        text = str(text)
    
    soup = BeautifulSoup(text, "html.parser")
    # Remove code blocks if requested
    if remove_code:
        for code in soup.find_all(['code', 'pre']):
            code.decompose()
    
    text = soup.get_text(separator=" ", strip=True)
    # Clean up excessive whitespace
    text = re.sub(r"\s+", " ", text).strip()
    # Remove urls
    text = re.sub(r"http\S+", "", text)

    # Remove '@' character using regex
    text = re.sub(r'@\w+', '', text)

    # Remove non-ASCII characters
    text = ''.join([char for char in text if ord(char) < 128])
    text = re.sub(r'\[|\]', '', text)
    
    text = text.split()
    text = [word for word in text if word.lower() not in intensifiers]
    text = " ".join(text)

    # Return the cleaned text as a string
    return text

In [8]:
def get_code(html_content):
    # Handle None or empty content
    if html_content is None or html_content == "" or (isinstance(html_content, float) and np.isnan(html_content)):
        return ""  # Return empty string for consistency

    # Parse HTML
    soup = BeautifulSoup(html_content, "html.parser")
    
    # Extract code blocks
    code_blocks = []
    for code in soup.find_all(['code', 'pre']):
        code_text = code.get_text(strip=True)
        if code_text:  # Only add non-empty code blocks
            code_blocks.append(code_text)
        # Remove code blocks from the soup to avoid duplication
        code.decompose()
    code_content = "\n---\n".join(code_blocks) if code_blocks else ""
    return code_content

In [9]:
def remove_stopwords(text):
    # Handle non-string inputs
    if not isinstance(text, str):
        return ""
    
    # Tokenize the text
    tokens = word_tokenize(text.lower())
    
    # Get English stopwords
    stop_words = set(stopwords.words('english'))
    
    # Filter out stopwords
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
    
    # Join tokens back into a string
    return ' '.join(filtered_tokens)

In [11]:
answer = df['AcceptedAnswerBody'].loc[24]
len(answer)

1020

In [12]:
df['Question_Code'] = df['Body'].apply(get_code)
df['Answer_Code'] = df['AcceptedAnswerBody'].apply(get_code)

In [13]:
df['Title_Clean'] = df['Title'].apply(clean_text) 
df['Body_Clean'] = df['Body'].apply(clean_text)
df['AcceptedAnswerBody_Clean'] = df['AcceptedAnswerBody'].apply(clean_text)

In [14]:
df['combination_text'] = df['Title_Clean'] + " " + df['Body_Clean'] + " " +  df['AcceptedAnswerBody_Clean']

In [15]:
# Test with remove stop word for all combination text
df['combination_text_no_stopw'] = df['combination_text'].apply(remove_stopwords)

In [16]:
'''parameter:
- data will be a single dataframe columns where we want to combine all the text and provide wordcloud
- Title will be the output name of wordcloud make sure it's meaningful'''
def wc_generating(data,title):
    #Define the model
    stopwords = STOPWORDS
    wc = WordCloud(
        background_color='white',
        stopwords=stopwords,
        height=600,
        width=400
    )
    #Combine all text of each rows to big text
    all_text = ' '.join(data.fillna(''))
    wc.generate(all_text)
    wc.to_file(f'{title}.png')

In [ ]:
wc_generating(df['Title_Clean'],"First Attempt for Title")
wc_generating(df['combination_text_no_stopw'],"First Attempt Combination with no stopword")

#### Can do more text cleaning later

## Strategy:

- Remove some intensifier words (from Assignment1), noise word (could use wordcloud to detect them, define some lists)
- Clustering them by embedding first then DBSCAN
- Then could try regex pattern

In [22]:
from sentence_transformers import SentenceTransformer
# 1. Load a pretrained Sentence Transformer model
model_embedding = SentenceTransformer("all-MiniLM-L6-v2")

In [23]:
from sklearn.cluster import KMeans,DBSCAN
def cluster_embedding(dframe,column,model_embedding,cluster_type="kmean",):
    embeddings_title = model_embedding.encode(dframe[column].tolist())
    dbscan = DBSCAN(eps=0.5,min_samples=5,metric='euclidean')
    k = 10  # or whatever value you want to test
    kmeans = KMeans(n_clusters=k, random_state=42)
    if cluster_type == "kmean":
        cluster_kmean = kmeans.fit_predict(embeddings_title)
        dframe['cluster_kmean'] = cluster_kmean
    elif cluster_type == "dbscan":
        clusters_dbscan = dbscan.fit_predict(embeddings_title)
        dframe['cluster_kmean'] = cluster_kmean
    else:
        cluster_kmean = kmeans.fit_predict(embeddings_title)
        dframe['cluster_kmean'] = cluster_kmean
        clusters_dbscan = dbscan.fit_predict(embeddings_title)
        dframe['cluster_kmean'] = cluster_kmean
    
        

In [24]:
cluster_embedding(df,'Title_Clean',model_embedding)

,QuestionId,Title,Body,CreationDate,Score,ViewCount,AnswerCount,AllTags,AcceptedAnswerId,AcceptedAnswerBody,Question_Code,Answer_Code,Title_Clean,Body_Clean,AcceptedAnswerBody_Clean,combination_text,combination_text_no_stopw,cluster_kmean
0,79501178,Store images instead of showing in a server,<p>I am running the code found on this [site][...,2025-03-11 14:50:31,0,23,1,<python><nlp><large-language-model>,79501337.0,<p>I can't test it but ...</p>\n<p>I checked <...,server\n---\nSSH\n---\nskip_tokens = [1] # sk...,"matplotlib\n---\nshow=True\n---\nfig, ax\n---\...",Store images instead of showing in a server,I am running the code found on this site1 in m...,I can't test it but ... I checked source code ...,Store images instead of showing in a server I ...,store images instead showing server running co...,8
1,79498915,Comparing the similarity of spoken and written...,<p>I'm converting spoken form text to its writ...,2025-03-10 18:55:59,0,20,1,<nlp><large-language-model>,NaN,NaN,,,Comparing the similarity of spoken and written...,I'm converting spoken form text to its written...,,Comparing the similarity of spoken and written...,comparing similarity spoken written form text ...,3
2,79488426,Upserting in Pinecone takes too long,<p>I'm trying to upsert reviews that i've scra...,2025-03-06 06:22:35,1,37,1,<python><nlp><rag><pinecone>,NaN,NaN,jina-embedding-v3\n---\nif index_name not in p...,,Upserting in Pinecone takes too long,I'm trying to upsert reviews that i've scraped...,,Upserting in Pinecone takes too long I'm tryin...,upserting pinecone takes long 'm trying upsert...,8
3,79484448,How does ELMo generate words for training ? Is...,<p>I'm confused about using Bidirectional LM f...,2025-03-04 17:32:14,0,28,1,<nlp><language-model><autoregressive-models><e...,NaN,NaN,,,How does ELMo generate words for training ? Is...,I'm confused about using Bidirectional LM for ...,,How does ELMo generate words for training ? Is...,elmo generate words training ? autoregressive ...,7
4,79482290,How to handle German language specific charact...,"<p>I am working with German Texts, where I nee...",2025-03-03 22:32:36,1,59,1,<python><nlp><tokenize><large-language-model><...,NaN,NaN,from transformers import GPT2Tokenizer\n\ntext...,,How to handle German language specific charact...,"I am working with German Texts, where I need t...",,How to handle German language specific charact...,handle german language specific characters lik...,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16674,62328,Is there an algorithm that tells the semantic ...,"<p>input: phrase 1, phrase 2</p>\n\n<p>output:...",2008-09-15 12:26:42,65,49872,11,<algorithm><nlp><semantics>,63076.0,<hr>\n\n<p>You might want to check out this pa...,,,Is there an algorithm that tells the semantic ...,"input: phrase 1, phrase 2 output: semantic sim...",You might want to check out this paper: Senten...,Is there an algorithm that tells the semantic ...,algorithm tells semantic similarity two phrase...,3
16675,41424,"How do you implement a ""Did you mean""?",<blockquote>\n <p><strong>Possible Duplicate:...,2008-09-03 10:36:13,118,33149,17,<nlp>,41448.0,<p>Actually what Google does is very much non-...,<spell_checked_word>,,"How do you implement a ""Did you mean""?",Possible Duplicate: How does the Google Did yo...,Actually what Google does is much non-trivial ...,"How do you implement a ""Did you mean""? Possibl...",implement `` mean '' ? possible duplicate : go...,8
16676,36533,Vista speech recognition in multiple languages,"<p>my primary language is spanish, but I use a...",2008-08-31 01:08:48,3,5658,6,<windows-vista><nlp><speech-recognition><multi...,36684.0,"<p>Citation from Vista <a href=""http://blogs.m...",,,Vista speech recognition in multiple languages,"my primary language is spanish, but I use all ...",Citation from Vista speech recognition blog : ...,Vista speech recognition in multiple languages...,vista speech recognition multiple languages pr...,8
16677,25

In [ ]:
embeddings_title = model.encode(df['Title_Clean'].tolist())

In [ ]:
from sklearn.cluster import DBSCAN
dbscan = DBSCAN(eps=0.5,min_samples=5,metric='euclidean')
clusters = dbscan.fit_predict(embeddings_title)

In [ ]:
df['cluster'] = clusters

In [19]:
df

,QuestionId,Title,Body,CreationDate,Score,ViewCount,AnswerCount,AllTags,AcceptedAnswerId,AcceptedAnswerBody,Question_Code,Answer_Code,Title_Clean,Body_Clean,AcceptedAnswerBody_Clean,combination_text,combination_text_no_stopw
0,79501178,Store images instead of showing in a server,<p>I am running the code found on this [site][...,2025-03-11 14:50:31,0,23,1,<python><nlp><large-language-model>,79501337.0,<p>I can't test it but ...</p>\n<p>I checked <...,server\n---\nSSH\n---\nskip_tokens = [1] # sk...,"matplotlib\n---\nshow=True\n---\nfig, ax\n---\...",Store images instead of showing in a server,I am running the code found on this site1 in m...,I can't test it but ... I checked source code ...,Store images instead of showing in a server I ...,store images instead showing server running co...
1,79498915,Comparing the similarity of spoken and written...,<p>I'm converting spoken form text to its writ...,2025-03-10 18:55:59,0,20,1,<nlp><large-language-model>,NaN,NaN,,,Comparing the similarity of spoken and written...,I'm converting spoken form text to its written...,,Comparing the similarity of spoken and written...,comparing similarity spoken written form text ...
2,79488426,Upserting in Pinecone takes too long,<p>I'm trying to upsert reviews that i've scra...,2025-03-06 06:22:35,1,37,1,<python><nlp><rag><pinecone>,NaN,NaN,jina-embedding-v3\n---\nif index_name not in p...,,Upserting in Pinecone takes too long,I'm trying to upsert reviews that i've scraped...,,Upserting in Pinecone takes too long I'm tryin...,upserting pinecone takes long 'm trying upsert...
3,79484448,How does ELMo generate words for training ? Is...,<p>I'm confused about using Bidirectional LM f...,2025-03-04 17:32:14,0,28,1,<nlp><language-model><autoregressive-models><e...,NaN,NaN,,,How does ELMo generate words for training ? Is...,I'm confused about using Bidirectional LM for ...,,How does ELMo generate words for training ? Is...,elmo generate words training ? autoregressive ...
4,79482290,How to handle German language specific charact...,"<p>I am working with German Texts, where I nee...",2025-03-03 22:32:36,1,59,1,<python><nlp><tokenize><large-language-model><...,NaN,NaN,from transformers import GPT2Tokenizer\n\ntext...,,How to handle German language specific charact...,"I am working with German Texts, where I need t...",,How to handle German language specific charact...,handle german language specific characters lik...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16674,62328,Is there an algorithm that tells the semantic ...,"<p>input: phrase 1, phrase 2</p>\n\n<p>output:...",2008-09-15 12:26:42,65,49872,11,<algorithm><nlp><semantics>,63076.0,<hr>\n\n<p>You might want to check out this pa...,,,Is there an algorithm that tells the semantic ...,"input: phrase 1, phrase 2 output: semantic sim...",You might want to check out this paper: Senten...,Is there an algorithm that tells the semantic ...,algorithm tells semantic similarity two phrase...
16675,41424,"How do you implement a ""Did you mean""?",<blockquote>\n <p><strong>Possible Duplicate:...,2008-09-03 10:36:13,118,33149,17,<nlp>,41448.0,<p>Actually what Google does is very much non-...,<spell_checked_word>,,"How do you implement a ""Did you mean""?",Possible Duplicate: How does the Google Did yo...,Actually what Google does is much non-trivial ...,"How do you implement a ""Did you mean""? Possibl...",implement `` mean '' ? possible duplicate : go...
16676,36533,Vista speech recognition in multiple languages,"<p>my primary language is spanish, but I use a...",2008-08-31 01:08:48,3,5658,6,<windows-vista><nlp><speech-recognition><multi...,36684.0,"<p>Citation from Vista <a href=""http://blogs.m...",,,Vista speech recognition in multiple languages,"my primary language is spanish, but I use all ...",Citation from Vista speech recognition blog : ...,Vista speech recognition in multiple languages...,vista speech recognition multiple languages pr...
16677,25332,What's a good natural language

In [20]:


k = 10  # or whatever value you want to test
kmeans = KMeans(n_clusters=k, random_state=42)
cluster_kmean = kmeans.fit_predict(embeddings_title)

df['cluster_kmean'] = cluster_kmean


NameError: name 'KMeans' is not defined

In [49]:
df[['Title_Clean','cluster','cluster_kmean']]

,Title_Clean,cluster,cluster_kmean
0,Store images instead of showing in a server,-1,8
1,Comparing the similarity of spoken and written...,-1,3
2,Upserting in Pinecone takes too long,-1,8
3,How does ELMo generate words for training ? Is...,-1,7
4,How to handle German language specific charact...,-1,9
...,...,...,...
16674,Is there an algorithm that tells the semantic ...,-1,3
16675,"How do you implement a ""Did you mean""?",-1,8
16676,Vista speech recognition in multiple languages,-1,8
16677,What's a good natural language library to use ...,-1,3


### Define Tags:

Plan to get tags?

- Keep filtering the Word Cloud and then manually create 10 topics?

: first to identify the words that better represent a certain document
second to encode them into vectors
BERT-based keyword extraction algorithm
KeyBERT [20], and a pre-defined dictionary of
topic-keywords developed by the team
The computation of semantic text similarity is performed after encoding the
keywords with Sentence-BERT

The first
stream of methods include solutions such as a set of logical rules that map words to topics
or comparison with a user defined taxonomy or ontology

The long-standing Latent Semantic Analysis (LSA) [5] and Latent Dirichlet Allocation (LDA) [3] models are well
suited to perform information reduction and exploratory analysis tasks
    However, this unsupervised approach has a few drawbacks: topic models might be unstable when not optimized [1] and their outputs might often be difficult to understand [4], as each topic corresponds with a combination of words which need to be interpreted by the user
    Another downside is the fact that the researcher needs to make assumptions on the number of topics to be retained from a certain collection

    Some scholars tried to mitigate all these issues by developing semi-supervised models that include ”anchor words” [7], or using partial labeling strategies 

KeyBERT

Create a keyword dict, I can start using word cloud for this to do 1 by one

In [ ]:
'''Define a function to removing some noise word
This will return a new column in dataframe the text that remove defined noise word'''

def remove_noise_word(text,noise_words):
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    words = text.split()
    # Remove noise words
    filtered = [word for word in words if word.lower() not in noise_words]
    return " ".join(filtered)

In [51]:
'''Define function to lematize can work parallel'''
# Lemmatization function
def lemma_texts_parallel(texts): #code is help by AI - Reference (4)
    cleaned_texts_lemma = []
    # Lemmatization with spaCy using parallel processing
    for doc in nlp.pipe(texts, batch_size=50, n_process=4):  # Process in parallel
        cleaned_tokens = [
            token.lemma_ for token in doc
        ]
        cleaned_texts_lemma.append(" ".join(cleaned_tokens))  # Join cleaned words back
    
    return pd.Series(cleaned_texts_lemma)

In [ ]:
noise_words1 = {'word','text','python','string','sentence','model'}
df['Title_Clean_No_Noise1'] = df['Title_Clean'].apply(lambda x: remove_noise_word(x, noise_words1))

In [49]:
wc_generating(df['Title_Clean_No_Noise1'],"No Noise Word 1")

In [55]:
noise_words2 = {'word','words','list','natural language','nlp','text','python','string','sentence','model'}
df['Title_Clean_No_Noise2'] = df['Title_Clean'].apply(lambda x: remove_noise_word(x, noise_words2))

In [56]:
wc_generating(df['Title_Clean_No_Noise2'],"No Noise Word 2")

In [ ]:
df['Lemma_Title'] = lemma_texts_parallel(df['Title_Clean'].tolist(remove_noise_word))

In [60]:
df['Title_Clean_No_Noise2'].to_csv("title_clean_remove_noise.csv", index=False)

In [ ]:
df['Title_Clean_No_Noise2'] = df['Title_Clean_No_Noise2'].apply(remove_stopwords)

In [68]:
from keybert import KeyBERT
kw_model = KeyBERT()

In [73]:
def extract_keyword(text):
    # Extract keywords using KeyBert; returns list of (keyword, score)
    keyword_tuples = kw_model.extract_keywords(
        text, 
        keyphrase_ngram_range=(1, 2),
        use_maxsum=True, 
        nr_candidates=10, 
        top_n=5
    )
    # Extract only the keyword strings from the tuples
    keywords = [kw for kw, score in keyword_tuples]
    return ",".join(keywords)

In [74]:
df['keywords_fromBert'] = df['Title_Clean_No_Noise2'].apply(extract_keyword)

In [77]:
df[['keywords_fromBert','cluster_kmean']].to_csv("keywords_fromBert.csv", index=False)

In [80]:
for_category = (
    df[['keywords_fromBert', 'cluster_kmean']]
    .replace('', np.nan)    # Replace empty strings with NaN
    .dropna(axis=0)        # Now drop rows that have NaN
)

In [100]:
for_category['keywords_list'] = for_category['keywords_fromBert'].str.split(',') 
cluster_group = for_category['cluster_kmean'].unique()
cluster_group.sort()

pattern_dfs = []
def get_frequency(data,min_support=0.01):
    te = TransactionEncoder()
    item_te = te.fit(data).transform(data)
    df_items_1hot = pd.DataFrame(item_te, columns=te.columns_)
    frequent_itemsets_apriori = apriori(df_items_1hot, min_support=min_support, use_colnames=True)
    return frequent_itemsets_apriori

for cluster in cluster_group:
    cluster_data = for_category.loc[for_category['cluster_kmean']==cluster,'keywords_list']
    pattern =  get_frequency(cluster_data)
    pattern = pattern.loc[pattern['itemsets'].apply(lambda x:len(x) >= 2)]
    # Add a new column for the cluster
    pattern['cluster'] = cluster
    pattern_dfs.append(pattern)
    
# Concatenate all clusters into one dataframe
frequency_df = pd.concat(pattern_dfs, ignore_index=True)
# Rename columns as required: score, pattern, and cluster
frequency_df.rename(columns={'support': 'score', 'itemsets': 'pattern'}, inplace=True)
    

In [101]:
frequency_df

,score,pattern,cluster
0,0.025666,"(spacy, custom)",0
1,0.013820,"(spacy, dependency)",0
2,0.010859,"(doc, spacy)",0
3,0.010859,"(spacy, document)",0
4,0.010859,"(entities, ner)",0
...,...,...,...
75,0.010270,"(using word2vec, word2vec)",5
76,0.019255,"(vector, word2vec)",5
77,0.015404,"(vectors, word2vec)",5
78,0.010270,"(using word2vec, using, word2vec)",5


### Try and compare with BERT topic

In [102]:
from bertopic import BERTopic

In [104]:
# Initialize and fit the BERTopic model
docs = df['Title_Clean_No_Noise2'].tolist()
topic_model = BERTopic()
topics, probs = topic_model.fit_transform(docs)

In [112]:
# Get a summary of topics
topic_info = topic_model.get_topic_info()
topic_info

,Topic,Count,Name,Representation,Representative_Docs
0,-1,4872,-1_find_nltk_language_file,"[find, nltk, language, file, sentences, error,...","[identify strings two different lists, compare..."
1,0,411,0_pandas_dataframe_column_frame,"[pandas, dataframe, column, frame, columns, ro...","[implement function pandas dataframe column, p..."
2,1,344,1_bert_bertmodel_embeddings_fine,"[bert, bertmodel, embeddings, fine, finetuning...","[language training bert, bert classification, ..."
3,2,285,2_sentiment_analysis_reviews_aspect,"[sentiment, analysis, reviews, aspect, negativ...",[sentiment analysis object attribute sentiment...
4,3,262,3_word2vec_text2vec_vectors_pretrained,"[word2vec, text2vec, vectors, pretrained, vect...","[using word2vec embedding sentences, using wor..."
...,...,...,...,...,...
270,269,10,269_ner_rulers_identity_types,"[ner, rulers, identity, types, entities, overa...",[large difference overall f score custom spacy...
271,270,10,270_layoutlm_javamodel_falcon7b40b_simpletrans...,"[layoutlm, javamodel, falcon7b40b, simpletrans...","[prepare custom training data layoutlm, input ..."
272,271,10,271_java_classifiers_svmhmm_program,"[java, classifiers, svmhmm, program, virtual, ...","[classification java, using multiple classifie..."
273,272,10,272_cuda_colab_usecuda_torchoutofmemoryerror,"[cuda, colab, usecuda, torchoutofmemoryerror, ...","[running process error saying cuda memory, err..."


Source:
-https://medium.com/@davidlfliang/intro-getting-started-with-text-embeddings-using-bert-9f8c3b98dee6
- https://www.sciencedirect.com/science/article/pii/S1877050922008766

Differences between the two algorithms:

DBSCAN is a density-based clustering algorithm, whereas K-Means is a centroid-based clustering algorithm.
DBSCAN can discover clusters of arbitrary shapes, whereas K-Means assumes that the clusters are spherical.
DBSCAN does not require the number of clusters to be specified in advance, whereas K-Means requires the number of clusters to be specified.
DBSCAN is less sensitive to initialization than K-Means.
When to use DBSCAN vs. K-Means?

Use DBSCAN when the data has irregular shapes or when there is no prior knowledge about the number of clusters.
Use K-Means when the data has spherical shapes and when the number of clusters is known beforehand.
If you are unsure which algorithm to use, it is always a good idea to try both algorithms and compare their results.